### Import Libraries

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models, transforms
from sklearn.metrics import f1_score
from PIL import Image
from tqdm import tqdm
from transformers import (ViTModel, 
                          ViTConfig, 
                          ViTFeatureExtractor, 
                          ViTForImageClassification)
from config import CFG
from sklearn.model_selection import StratifiedKFold

2024-01-07 19:50:53.161562: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Load data

In [2]:
label_data = pd.read_csv(os.path.join(CFG.train_data_dir, '文件标签汇总数据.csv'))
train_csv_folder = os.path.join(CFG.train_data_dir, 'csv文件')
test_A_csv_folder = os.path.join(CFG.test_A_data_dir, 'csv文件')
train_image_folder = os.path.join(CFG.project_name, 'project/image/训练集数据')
test_A_image_folder = os.path.join(CFG.project_name, 'project/image/A榜测试集数据')

In [3]:
class TrainDataset(Dataset):
    def __init__(self, image_folder, transform=None):
        self.image_folder = image_folder
        self.transform = transform
        self.image_files = [f for f in os.listdir(
            image_folder) if f.endswith('.png')]
        # 假设标签以某种方式存储或从文件名中获取
        self.labels = [CFG.label2id[f[0]] for f in self.image_files]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_folder, self.image_files[idx])
        image = Image.open(img_name)
        # if image.mode != 'RGB':
        #     image = image.convert('RGB')

        label = CFG.label2id[self.image_files[idx][0]]

        if self.transform:
            image = self.transform(image)

        return image, label

In [4]:
class TestDataset(Dataset):
    def __init__(self, image_folder, transform=None):
        self.image_folder = image_folder
        self.transform = transform
        self.image_files = [f for f in os.listdir(
            image_folder) if f.endswith('.png')]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_folder, self.image_files[idx])
        image = Image.open(img_name)
        # if image.mode != 'RGB':
        #     image = image.convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, self.image_files[idx]

In [5]:
data_transforms = {
    'train': transforms.Compose([  
        transforms.RandomHorizontalFlip(),  # 随机水平翻转
        transforms.RandomVerticalFlip(),    # 随机垂直翻转
        transforms.RandomRotation(10),      # 在[-10, 10]范围内随机旋转
        transforms.RandomHorizontalFlip(),
        transforms.RandomResizedCrop(224),
        transforms.ToTensor(),
        transforms.Lambda(lambda x: x.repeat(3, 1, 1)),
        transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
    ]),
    'test': transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Lambda(lambda x: x.repeat(3, 1, 1)),
        transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
    ]),
}

In [6]:
train_dataset = TrainDataset(
    image_folder=train_image_folder, transform=data_transforms['train'])
test_dataset = TestDataset(
    image_folder=test_A_image_folder, transform=data_transforms['test'])

train_loader = DataLoader(train_dataset, batch_size=4,
                          shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=4,
                         shuffle=False, num_workers=4)

dataset_sizes = {'train': len(train_dataset), 'test': len(test_dataset)}

In [7]:
model = ViTForImageClassification.from_pretrained(CFG.vit_model,
                                                    num_labels=5,
                                                    ignore_mismatched_sizes=True)
model.to(CFG.device)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at /ai/users/bst/competition/model/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([5]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([5, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=7

In [8]:
from config import CFG
# 用K折交叉验证，对train_dataset进行划分, 5折,并且评估指标是f1_score
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=CFG.seed)
for fold, (train_index, valid_index) in enumerate(skf.split(train_dataset.image_files, train_dataset.labels)):
    train_dataset_fold = torch.utils.data.Subset(train_dataset, train_index)
    valid_dataset_fold = torch.utils.data.Subset(train_dataset, valid_index)
    train_loader_fold = DataLoader(train_dataset_fold, batch_size=8,
                                   shuffle=True, num_workers=4)
    valid_loader_fold = DataLoader(valid_dataset_fold, batch_size=8,
                                   shuffle=False, num_workers=4)
    dataset_sizes = {'train': len(train_dataset_fold), 'valid': len(valid_dataset_fold)}
    print(f'fold:{fold}')
    # print(f'train_index:{train_index}')
    # print(f'valid_index:{valid_index}')
    print('--------------------------------')

    # 用于训练的模型
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0001)

    # 训练
    for epoch in range(CFG.epochs):
        print('Epoch {}/{}'.format(epoch+1, CFG.epochs))
        print('-' * 10)

        # 每个epoch都有一个训练和验证阶段
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()  # 设置模型为训练模式
                dataloader = train_loader_fold
            else:
                model.eval()   # 设置模型为评估模式
                dataloader = valid_loader_fold

            running_loss = 0.0
            running_corrects = 0
            y_true = []
            y_pred = []
            # 迭代数据
            for inputs, labels in tqdm(dataloader):
                inputs = inputs.to(CFG.device)
                labels = labels.to(CFG.device)

                # 零参数梯度
                optimizer.zero_grad()

                # 前向
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs.logits, 1)
                    loss = criterion(outputs.logits, labels)
                    # 后向+仅在训练阶段进行优化
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                # print(f'labels:{labels}')
                # print(f'preds:{preds}')
                # 统计f1_score
                y_true.extend(labels.cpu().numpy().tolist())
                y_pred.extend(preds.cpu().numpy().tolist())
                # 统计loss
                running_loss += loss.item() * inputs.size(0)
            # 分别计算5个label的f1_score,然后加权平均[0.1, 0.3, 0.2, 0.1, 0.3]
            score = f1_score(y_true, y_pred, average=None)
            score = np.average(score, weights=[0.1, 0.3, 0.2, 0.1, 0.3])
            # 计算loss
            epoch_loss = running_loss / dataset_sizes[phase]
            print('{} Loss: {:.4f} F1-score: {:.4f}'.format(
                phase, epoch_loss, score))
    # 保存模型
    torch.save(model.state_dict(), f'./model/vit_model_{fold}.pth')
    print(f'./model/vit_model_{fold}.pth saved')



fold:0
--------------------------------
Epoch 1/25
----------


100%|██████████| 12/12 [00:04<00:00,  2.89it/s]


train Loss: 2174309.4891 F1-score: 0.0441


100%|██████████| 3/3 [00:00<00:00,  4.82it/s]


valid Loss: 2108420.7717 F1-score: 0.0467
Epoch 2/25
----------


...

valid Loss: 2108237.5109 F1-score: 0.0467
Epoch 25/25
----------


100%|██████████| 12/12 [00:02<00:00,  4.54it/s]


train Loss: 2174120.4022 F1-score: 0.0441


100%|██████████| 3/3 [00:00<00:00,  4.72it/s]


valid Loss: 2108235.6196 F1-score: 0.0467
./model/vit_model_4.pth saved


In [9]:
model.eval()
predictions = []
file_name = []
for images,img_name in test_loader:
    with torch.no_grad():
        images = images.to(CFG.device)  
        outputs = model(images)
        _, predicted =torch.max(outputs.logits, 1)
        predictions.extend(predicted.cpu().numpy())
        file_name.extend(list(img_name))

RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/ai/anaconda3/envs/kaggle/lib/python3.8/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
  File "/ai/anaconda3/envs/kaggle/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 54, in fetch
    return self.collate_fn(data)
  File "/ai/anaconda3/envs/kaggle/lib/python3.8/site-packages/torch/utils/data/_utils/collate.py", line 265, in default_collate
    return collate(batch, collate_fn_map=default_collate_fn_map)
  File "/ai/anaconda3/envs/kaggle/lib/python3.8/site-packages/torch/utils/data/_utils/collate.py", line 142, in collate
    return [collate(samples, collate_fn_map=collate_fn_map) for samples in transposed]  # Backwards compatibility.
  File "/ai/anaconda3/envs/kaggle/lib/python3.8/site-packages/torch/utils/data/_utils/collate.py", line 142, in <listcomp>
    return [collate(samples, collat

In [ ]:
csv_file_name = []
test_A_csv_file_name = os.listdir(test_A_csv_folder)
for file in file_name:
    for j in test_A_csv_file_name:
        if file[:-4] in j:
            csv_file_name.append(j)
            break

In [ ]:
label = [CFG.id2label[i] for i in predictions]

In [ ]:
results_df = pd.DataFrame({'defectType': label,'fileName': csv_file_name})

In [ ]:
results_df.to_csv('submission.csv', index=False)

In [ ]:
train_dataset[0][0].shape

torch.Size([1, 224, 224])